In [ ]:
!pip install yolov4==2.1.0

  Created wheel for yolov4: filename=yolov4-2.1.0-cp37-none-any.whl size=39936 sha256=cb162cba769333a02ab3e3a0d3f1036f7fc521bb82ea97e39d8518a4bbb34309
  Stored in directory: /root/.cache/pip/wheels/7e/14/b8/fa302e43edc362a5cc332216fcda60d8f7742958308379f71e
Successfully built yolov4


In [ ]:
import os

with open('/content/drive/MyDrive/MultiDetectionTraining/damage/train.txt', 'w') as out:
  for img in [f for f in os.listdir('/content/drive/MyDrive/MultiDetectionTraining/damage/train') if f.endswith('jpg')]:
    out.write('/content/drive/MyDrive/MultiDetectionTraining/obj/' + img + '\n')
  out.close()

In [ ]:
import os

with open('/content/drive/MyDrive/MultiDetectionTraining/damage/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('/content/drive/MyDrive/MultiDetectionTraining/damage/valid') if f.endswith('jpg')]:
    out.write('/content/drive/MyDrive/MultiDetectionTraining/obj/' + img + '\n')
  out.close()

In [ ]:
import tensorflow
physicalDevices = tensorflow.config.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(physicalDevices[0], True)

In [ ]:
# Object Detection Training

from tensorflow.keras import callbacks, optimizers
from yolov4.tf import SaveWeightsCallback, YOLOv4
import time

yolo = YOLOv4()
yolo.classes = "/content/drive/MyDrive/MultiDetectionTraining/obj.names"
yolo.input_size = (640, 480)
yolo.batch_size = 4

yolo.make_model()
yolo.load_weights(
    "/content/drive/MyDrive/MultiDetectionTraining/yolov4.conv.137",
    weights_type="yolo"
    )

train_data_set = yolo.load_dataset(
    "/content/drive/MyDrive/MultiDetectionTraining/damage/train.txt",
    dataset_type="yolo",
    image_path_prefix="",
    label_smoothing=0.05
)

val_data_set = yolo.load_dataset(
    "/content/drive/MyDrive/MultiDetectionTraining/damage/valid.txt",
    dataset_type="yolo",
    image_path_prefix="",
    training=False
)

epochs = 200
lr = 1e-4

optimizer = optimizers.Adam(learning_rate=lr)
yolo.compile(optimizer=optimizer, loss_iou_type="ciou")

def lr_scheduler(epoch):
    if epoch < int(epochs * 0.5):
        return lr
    if epoch < int(epochs * 0.8):
        return lr * 0.5
    if epoch < int(epochs * 0.9):
        return lr * 0.1
    return lr * 0.01

_callbacks = [
    callbacks.LearningRateScheduler(lr_scheduler),
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        log_dir="/content/drive/MyDrive/MultiDetectionTraining/logs",
    ),
    SaveWeightsCallback(
        yolo=yolo, dir_path="/content/drive/MyDrive/MultiDetectionTraining/trained",
        weights_type="yolo", epoch_per_save=10
    ),
]

yolo.fit(
    train_data_set,
    epochs=epochs,
    callbacks=_callbacks,
    validation_data=val_data_set,
    validation_steps=100,
    validation_freq=5,
    steps_per_epoch=100,
)

Streaming output truncated to the last 5000 lines.
grid: 30*40 iou_loss: 0.0767583773 conf_loss: 0.0546504185 prob_loss: 5.96047187e-07 total_loss 0.131409392
grid: 15*20 iou_loss: 0.286719233 conf_loss: 0.0390013307 prob_loss: 1.03414577e-05 total_loss 0.32573092
grid: 60*80 iou_loss: 0.0461077876 conf_loss: 0.0290912427 prob_loss: 0 total_loss 0.0751990303
grid: 30*40 iou_loss: 0.815849125 conf_loss: 0.0600292794 prob_loss: 0.000215807435 total_loss 0.876094222
grid: 15*20 iou_loss: 0.338382661 conf_loss: 0.0387581512 prob_loss: 5.69225722e-06 total_loss 0.377146512
grid: 60*80 iou_loss: 0 conf_loss: 0.0235186201 prob_loss: 0 total_loss 0.0235186201
grid: 30*40 iou_loss: 0.348076671 conf_loss: 0.0774890929 prob_loss: 1.13547567e-05 total_loss 0.425577104
grid: 15*20 iou_loss: 0.590604067 conf_loss: 0.0244883504 prob_loss: 1.02818358e-05 total_loss 0.615102708
grid: 60*80 iou_loss: 0 conf_loss: 0.0545386598 prob_loss: 0 total_loss 0.0545386598
grid: 30*40 iou_loss: 0.724464774 conf_lo

In [ ]:
from yolov4.tf import YOLOv4

yolo = YOLOv4()

yolo.classes = "/content/drive/MyDrive/MultiDetectionTraining/obj.names"

yolo.make_model()
yolo.load_weights("/content/drive/MyDrive/MultiDetectionTraining/trained/yolov4-final.weights", weights_type="yolo")

dataset = yolo.load_dataset(
    "/content/drive/MyDrive/MultiDetectionTraining/damage/valid.txt",
    dataset_type="yolo",
    training=False,
    image_path_prefix="",
)

yolo.save_dataset_for_mAP("/content/drive/MyDrive/MultiDetectionTraining/mAP", dataset)

In [ ]:
!python /content/drive/MyDrive/MultiDetectionTraining/mAP/main.py

9.81% = Damage AP 
mAP = 9.81%
<Figure size 640x480 with 1 Axes>
